## Requirements

In [1]:
import duckdb
import pandas as pd

## Database connection

Create a connection to the database, and query metadata.

In [2]:
conn = duckdb.connect('data/patient_experiment.csv')

In [3]:
conn.sql('''
    DESCRIBE patient_experiment;
''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ column0     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ patient     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ dose        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ date        │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

Create a function to show the tables/views in the database.

In [6]:
def show_tables(conn):
    conn.sql('''
        SELECT table_schema, table_name, table_type
          FROM information_schema.tables
          WHERE table_schema NOT IN ('information_schema', 'pg_catalog');
    ''').show()

In [7]:
show_tables(conn)

┌──────────────┬────────────────────┬────────────┐
│ table_schema │     table_name     │ table_type │
│   varchar    │      varchar       │  varchar   │
├──────────────┼────────────────────┼────────────┤
│ main         │ file               │ VIEW       │
│ main         │ patient_experiment │ VIEW       │
└──────────────┴────────────────────┴────────────┘



## Queries

Select all the data for patient 6 and convert it to a pandas dataframe.

In [18]:
conn.execute('''
    SELECT patient, date, temperature, dose
      FROM patient_experiment
      WHERE patient == 6;
''').df()

,patient,date,temperature,dose
0,6,2012-10-02 10:00:00,37.5,0.0
1,6,2012-10-02 11:00:00,38.1,2.0
2,6,2012-10-02 12:00:00,37.9,3.0
3,6,2012-10-02 13:00:00,37.7,2.0
4,6,2012-10-02 14:00:00,37.2,1.0
5,6,2012-10-02 15:00:00,36.8,0.0


For the patients with a high fever, count the number of timepoints they had a temperature above $39.5\textdegree C$ as well as their maximum temperature.

In [10]:
conn.execute('''
    SELECT
        patient,
        COUNT(temperature) AS high_fever_count,
        MAX(temperature) AS max_temperature
      FROM patient_experiment
      WHERE temperature > 39.5
      GROUP BY patient
      ORDER BY patient;
''').df()

,patient,high_fever_count,max_temperature
0,7,3,40.7
1,9,1,40.2


For each patient, compute the total dose administered, as well as the maximum temperature, and order by descending maximum temperature.

In [18]:
conn.execute('''
    SELECT
        patient,
        MAX(temperature) AS 'max_temperature',
        SUM(dose) AS 'total_dose'
      FROM patient_experiment
      GROUP BY patient
      ORDER BY max_temperature DESC;
''').df()

,patient,max_temperature,total_dose
0,7,40.7,30.0
1,9,40.2,30.0
2,5,39.5,27.0
3,3,39.5,13.0
4,2,39.4,15.0
5,1,38.5,6.0
6,6,38.1,8.0
7,4,38.1,10.0
8,8,37.9,0.0


If you want to query the result of such a query, you can create a view, `'hypothesis'` in this example.

In [11]:
conn.execute('''
    CREATE VIEW hypothesis AS SELECT
        patient,
        MAX(temperature) AS 'max_temperature',
        SUM(dose) AS 'total_dose'
      FROM patient_experiment
      GROUP BY patient
      ORDER BY max_temperature DESC;
''');

In [12]:
show_tables(conn)

┌──────────────┬────────────────────┬────────────┐
│ table_schema │     table_name     │ table_type │
│   varchar    │      varchar       │  varchar   │
├──────────────┼────────────────────┼────────────┤
│ main         │ file               │ VIEW       │
│ main         │ hypothesis         │ VIEW       │
│ main         │ patient_experiment │ VIEW       │
└──────────────┴────────────────────┴────────────┘



Get the maximum dose administered to a patient.

In [35]:
conn.execute('''
    SELECT MAX(total_dose)
      FROM hypothesis;
''').df()

,max(total_dose)
0,30.0


Although DuckDB has an extension to perform a pivot (this is not standard SQL), it is not as elegant as the pandas counterpart as multi-level columns are not suppported by DuckDB.

In [14]:
conn.execute('''
    CREATE TABLE time_series AS
      PIVOT patient_experiment
      ON patient
      USING
        first(temperature) AS temperature,
        first(dose) AS dose
      GROUP BY date;
''');

In [17]:
conn.sql('DESCRIBE time_series;').show()

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ date          │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ 1_temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 1_dose        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 2_temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 2_dose        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 3_temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 3_dose        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 4_temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 4_dose        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ 5_temperature │ DOUBLE      │ YES     │ NULL    │ NULL    │ NU

In [26]:
conn.execute('''
    SELECT
      date,
      "6_temperature" AS temperature,
      "6_dose" AS dose
    FROM time_series
    ORDER BY date;
''').df()

,date,temperature,dose
0,2012-10-02 10:00:00,37.5,0.0
1,2012-10-02 11:00:00,38.1,2.0
2,2012-10-02 12:00:00,37.9,3.0
3,2012-10-02 13:00:00,37.7,2.0
4,2012-10-02 14:00:00,37.2,1.0
5,2012-10-02 15:00:00,36.8,0.0
6,2012-10-02 16:00:00,NaN,NaN


In [27]:
conn.sql('SHOW TABLES;').show()

┌────────────────────┐
│        name        │
│      varchar       │
├────────────────────┤
│ file               │
│ hypothesis         │
│ patient_experiment │
│ time_series        │
└────────────────────┘



Create a view on a second CSV file.

In [40]:
conn.execute('''
   CREATE VIEW patient_metadata AS
       SELECT *
         FROM read_csv_auto('data/patient_metadata.csv', filename=true);
''');

In [41]:
show_tables(conn)

┌──────────────┬────────────────────┬────────────┐
│ table_schema │     table_name     │ table_type │
│   varchar    │      varchar       │  varchar   │
├──────────────┼────────────────────┼────────────┤
│ main         │ time_series        │ BASE TABLE │
│ main         │ file               │ VIEW       │
│ main         │ hypothesis         │ VIEW       │
│ main         │ metadata           │ VIEW       │
│ main         │ patient_experiment │ VIEW       │
│ main         │ patient_metadata   │ VIEW       │
└──────────────┴────────────────────┴────────────┘



Determine the patient IDs that are either in `patient_experiment`, or in `patient_metadata`, but not in both.  Note that a full outer join is used to combine the informantion in both tables.

In [45]:
conn.execute('''
    SELECT
      DISTINCT COALESCE(exp.patient, mt.patient) AS patient,
      CASE
        WHEN exp.patient IS NOT NULL AND mt.patient IS NULL
          THEN 'only in experiment'
        WHEN exp.patient IS NULL and mt.patient IS NOT NULL
          THEN 'only in metadata'
        ELSE 'in both'
      END AS present
      FROM patient_experiment AS exp FULL OUTER JOIN patient_metadata AS mt
        USING (patient)
      WHERE NOT present = 'in both'
      ORDER BY exp.patient, mt.patient;
''').df()

,patient,present
0,4,only in experiment
1,10,only in metadata
2,11,only in metadata


You can do an inner join between the tables `patient_experiment` and `patient_metadata` to get the maximum temperature, the condition and gender for each patient that occurs in both tables.

In [50]:
conn.execute('''
    SELECT
      COALESCE(exp.patient, mt.patient) AS patient,
      MAX(exp.temperature) AS max_temperature,
      ANY_VALUE(mt.condition) AS condition,
      ANY_VALUE(mt.gender) AS gender
    FROM patient_experiment AS exp INNER JOIN patient_metadata AS mt
      USING (patient)
    GROUP BY exp.patient, mt.patient
    ORDER BY exp.patient, mt.patient
''').df()

,patient,max_temperature,condition,gender
0,1,38.5,A,M
1,2,39.4,A,F
2,3,39.5,A,M
3,5,39.5,A,M
4,6,38.1,B,F
5,7,40.7,B,M
6,8,37.9,B,F
7,9,40.2,B,M


## New style versus classic style

In [51]:
conn.execute('''
    SELECT patient, date, temperature, dose
      FROM patient_experiment
      WHERE patient == 6;
''').df()

,patient,date,temperature,dose
0,6,2012-10-02 10:00:00,37.5,0.0
1,6,2012-10-02 11:00:00,38.1,2.0
2,6,2012-10-02 12:00:00,37.9,3.0
3,6,2012-10-02 13:00:00,37.7,2.0
4,6,2012-10-02 14:00:00,37.2,1.0
5,6,2012-10-02 15:00:00,36.8,0.0


In [52]:
conn.sql('''
    SELECT patient, date, temperature, dose
      FROM patient_experiment
''').filter('patient = 6').show()

┌─────────┬─────────────────────┬─────────────┬────────┐
│ patient │        date         │ temperature │  dose  │
│  int64  │      timestamp      │   double    │ double │
├─────────┼─────────────────────┼─────────────┼────────┤
│       6 │ 2012-10-02 10:00:00 │        37.5 │    0.0 │
│       6 │ 2012-10-02 11:00:00 │        38.1 │    2.0 │
│       6 │ 2012-10-02 12:00:00 │        37.9 │    3.0 │
│       6 │ 2012-10-02 13:00:00 │        37.7 │    2.0 │
│       6 │ 2012-10-02 14:00:00 │        37.2 │    1.0 │
│       6 │ 2012-10-02 15:00:00 │        36.8 │    0.0 │
└─────────┴─────────────────────┴─────────────┴────────┘



For the patients with a high fever, count the number of timepoints they had a temperature above $39.5\textdegree C$ as well as their maximum temperature.

In [10]:
conn.execute('''
    SELECT
        patient,
        COUNT(temperature) AS high_fever_count,
        MAX(temperature) AS max_temperature
      FROM patient_experiment
      WHERE temperature > 39.5
      GROUP BY patient
      ORDER BY patient;
''').df()

,patient,high_fever_count,max_temperature
0,7,3,40.7
1,9,1,40.2


In [73]:
conn.sql('SELECT patient, temperature FROM patient_experiment') \
    .filter('temperature > 39.5') \
    .aggregate(
        'patient, '
        'COUNT(temperature) AS high_fever_count, '
        'MAX(temperature) AS max_temperature',
        group_expr='patient') \
    .show()     

┌─────────┬──────────────────┬─────────────────┐
│ patient │ high_fever_count │ max_temperature │
│  int64  │      int64       │     double      │
├─────────┼──────────────────┼─────────────────┤
│       7 │                3 │            40.7 │
│       9 │                1 │            40.2 │
└─────────┴──────────────────┴─────────────────┘



For each patient, compute the total dose administered, as well as the maximum temperature, and order by descending maximum temperature.

In [18]:
conn.execute('''
    SELECT
        patient,
        MAX(temperature) AS 'max_temperature',
        SUM(dose) AS 'total_dose'
      FROM patient_experiment
      GROUP BY patient
      ORDER BY max_temperature DESC;
''').df()

,patient,max_temperature,total_dose
0,7,40.7,30.0
1,9,40.2,30.0
2,5,39.5,27.0
3,3,39.5,13.0
4,2,39.4,15.0
5,1,38.5,6.0
6,6,38.1,8.0
7,4,38.1,10.0
8,8,37.9,0.0


In [77]:
conn.sql('SELECT patient, temperature, dose from patient_experiment') \
    .aggregate(
        'patient, '
        'MAX(temperature) AS max_temperature, '
        'SUM(dose) AS total_dose',
        group_expr='patient'
    ) \
    .order('max_temperature DESC') \
    .show()

┌─────────┬─────────────────┬────────────┐
│ patient │ max_temperature │ total_dose │
│  int64  │     double      │   double   │
├─────────┼─────────────────┼────────────┤
│       7 │            40.7 │       30.0 │
│       9 │            40.2 │       30.0 │
│       5 │            39.5 │       27.0 │
│       3 │            39.5 │       13.0 │
│       2 │            39.4 │       15.0 │
│       1 │            38.5 │        6.0 │
│       4 │            38.1 │       10.0 │
│       6 │            38.1 │        8.0 │
│       8 │            37.9 │        0.0 │
└─────────┴─────────────────┴────────────┘



The new-style queries allow for lazy evaluation, while the classic-style queries are evaluated immediately.